In [ ]:
import json
from typing import List, Dict, Tuple
import numpy as np
import matplotlib.pyplot as plt
import seaborn

reservoir_sigma = [0.2, 0.4, 0.6, 0.8, 1.0]
rewiring = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
reservoir_degree = [2, 4, 6, 8]
spectral_radius = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0, 1.01, 1.02, 1.03, 1.04, 1.05]

data_path = '/home/dionisio35/Documents/thesis/outputs/MG16.8.json'

In [ ]:
def results_info_by_param(param_name: str, file: str):
    with open(file, 'r') as f:
        data = json.load(f)

    param_data = {}
    for x in data:
        current_param = x['params'][param_name]
        if param_data.get(current_param):
            param_data[current_param].append(x['index'])
        else:
            param_data[current_param] = [x['index']]
    
    return param_data

def results_info_by_params(params: List[str], file: str):
    with open(file, 'r') as f:
        data = json.load(f)

    param_data = {}
    for x in data:
        current_params = tuple((x['params'][a] for a in params))

        if param_data.get(current_params):
            param_data[current_params].append(x['index'])
        else:
            param_data[current_params] = [x['index']]
    
    new_data = {}
    for i in param_data:
        elements = np.array(param_data[i])
        new_data[i] = {'mean': elements.mean(), 'std': elements.std()}
    
    return new_data

# TODO
def generate_meshgrid(data: Dict[Tuple, float]):
    X = set()
    Y = set()
    for i in data.keys():
        X.add(i[0])
        Y.add(i[1])
    X, Y = np.meshgrid(sorted(list(X)), sorted(list(Y)))

    Z = []
    STD = []
    for x, y in zip(X, Y):
        z = []
        std = []
        for _x, _y in zip(x,y):
            z.append(data[(_x,_y)]['mean'])
            std.append(data[(_x,_y)]['std'])
        Z.append(z)
    Z = np.array(Z)
    
    return X, Y, Z, STD

def plot_3d(X, Y, Z, params: List[str]):
    fig = plt.figure(dpi=100)
    ax = plt.axes(projection ='3d')
    
    ax.plot_wireframe(X, Y, Z, cmap='viridis', edgecolor='green')
    
    for x, y, z in zip(X, Y, Z):
        for _x, _y, _z in zip(x, y, z):
            ax.plot([_x, _x], [_y, _y], [_z+0.1, _z-0.1], marker='_', color='red')

    ax.set_title('Title')
    ax.set_xlabel(params[0], fontsize=12)
    ax.set_ylabel(params[1], fontsize=12)
    ax.set_zlabel('z', fontsize=12)

def plot_2d(data, param: str):
    fig = plt.figure()
    ax = plt.axes()
    plt.plot(data)

    ax.set_title('Title')
    ax.set_xlabel(param, fontsize=12)
    ax.set_ylabel('threshold index', fontsize=12)


In [ ]:
data = results_info_by_params(['reservoir_degree'], data_path)

plot_2d([i['mean'] for i in data.values()], 'a')

In [ ]:
cases = [['reservoir_sigma', 'rewiring'], ['spectral_radius', 'rewiring'], ['reservoir_sigma', 'spectral_radius']]

for case in cases:
    data = results_info_by_params(case, data_path)
    X, Y, Z, STD = generate_meshgrid(data)
    plot_3d(X, Y, Z, case)